In [1]:
import csv
import pymongo
from pymongo import MongoClient
import datetime as dt
import calendar
import pandas as pd
import numpy as np
import time

In [2]:
Client = MongoClient("mongodb://3.20.189.28:2036/momoKashBd?readPreference=secondary&directConnection=true")
db_loan = Client["momoKashBd"]
#These are my collections
#Clients = db_v1["Client"]
#Loans = db_v1["Loan"]
Loans = db_loan["Loan"]
print("MongoDB connection established with success")

MongoDB connection established with success


In [3]:
#here we define the date to be used 
date_start = dt.datetime(2025, 4, 1)
date_end = dt.datetime(2025, 4, 30)

In [4]:
# Inspect one document to see field names and types
Loans.find_one()

{'_id': '237650000338:CAM07X01T20221104115211MU:4993969848',
 'loanID': 'CAM07X01T20221104115211MU',
 'loanAge': '0',
 'loanDate': '04/11/2022',
 'loanAmount': 5000.0,
 'autoDebitFlag': '0',
 'amountwithInterest': 5375.0,
 'amountDisbursed': 5000.0,
 'amountPaid': 702.0,
 'bonusAmount': 5250.0,
 'bankID': '3',
 'bonusDate': '07/11/2022',
 'bankName': 'CAMCCUL',
 'bonusInterest': 250.0,
 'carryoverDate': '04/12/2022',
 'clientID': '650000338',
 'clientLocation': '',
 'clientName': 'NATHAELLE GEORGIA MAYA BIKOE',
 'companyServiceID': '1',
 'dateTime': datetime.datetime(2022, 11, 4, 11, 52, 11),
 'dueDate': '19/11/2022',
 'dueAmount': 5077.0,
 'hadBonus': False,
 'interest_rate': 7.5,
 'is_complete': False,
 'isCarryover': True,
 'isPromo': False,
 'isReconductable': True,
 'isReconducted': False,
 'loanPolicyID': '1',
 'loanStatus': 'isCarryOver',
 'momotransaction_ID': '4993969848',
 'msisdn': '237650000338',
 'originalInterest': 375.0,
 'operatorProlongationDate': '03/01/2023',
 'promo

In [5]:
# Quick test: check if any loanDate exists
cursor = Loans.find(
    {"loanDate": {"$regex": "09/2025"}},
    {"loanID": 1, "loanDate": 1, "dateTime": 1}
)

for doc in cursor:
    print(doc)

{'_id': '237679441596:RB011X01T202509010000070129M79441596:13809463099', 'loanID': 'RB011X01T202509010000070129M79441596', 'loanDate': '01/09/2025', 'dateTime': datetime.datetime(2025, 9, 1, 0, 1, 40)}
{'_id': '237677501647:RB011X01T202509010000330402M77501647:13809464192', 'loanID': 'RB011X01T202509010000330402M77501647', 'loanDate': '01/09/2025', 'dateTime': datetime.datetime(2025, 9, 1, 0, 1, 47)}
{'_id': '237672023159:CCC024X01T202509010001460307M72023159:13809468619', 'loanID': 'CCC024X01T202509010001460307M72023159', 'loanDate': '01/09/2025', 'dateTime': datetime.datetime(2025, 9, 1, 0, 2, 22)}
{'_id': '237675603077:CCC024X01T202509010001540177M75603077:13809472472', 'loanID': 'CCC024X01T202509010001540177M75603077', 'loanDate': '01/09/2025', 'dateTime': datetime.datetime(2025, 9, 1, 0, 2, 56)}
{'_id': '237682877899:CCC024X01T202509010002260201M82877899:13809472947', 'loanID': 'CCC024X01T202509010002260201M82877899', 'loanDate': '01/09/2025', 'dateTime': datetime.datetime(2025, 9

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
# Cell 5 - Fetching raw loans from Mongo with Python-side date filter (using dateTime + status)

start = time.time()

# Fetch loans
cursor = Loans.find(
    { 
        "dateTime": {"$gte": date_start, "$lte": date_end},  # filtering by dateTime
    },
    {
        "_id": 0, 
        "loanAmount" : 1,
        "amountPaid" : 1,
        "dueDate" : 1,
        "loanDate" : 1,
        "dueAmount" :1,
        "amountwithInterest" : 1,
        "loanStatus" : 1,
        "interest_rate" : 1,
        "clientID" : 1, 
        "bonusAmount" : 1,
        "penaltyAmounnt": 1,
        "amountDisbursed" : 1,  
        "totalDueAmount" : 1,
        "dateTime": 1
    })

loans_df = pd.DataFrame(list(cursor))

#Check if empty
if loans_df.empty:
    print("No completed loans were made during this period.")
else:
    # Convert string date fields → datetime
    for col in ["loanDate", "dueDate"]:
        loans_df[col] = pd.to_datetime(loans_df[col], dayfirst=True, errors="coerce")

    # Sort refunds by dateTime
    loans_df = loans_df.sort_values("dateTime", ascending=True)

    print("Fetched completed loans:", loans_df.shape)
    print(loans_df.head(3))  # show a small sample

end = time.time()
print(f"Fetching time: {round((end-start)/60, 2)} minutes")        

Fetched completed loans: (170864, 13)
    loanDate  loanAmount  amountwithInterest  amountDisbursed  amountPaid  \
0 2025-04-01      1047.0              1126.0           1047.0         3.0   
1 2025-04-01      8000.0              8600.0           8000.0      9245.0   
2 2025-04-01     10000.0             10750.0          10000.0     11557.0   

   bonusAmount   clientID            dateTime    dueDate  dueAmount  \
0       1126.0  677688025 2025-04-01 00:00:30 2025-04-15     1208.0   
1       8600.0  672635655 2025-04-01 00:00:56 2025-04-16        0.0   
2      10750.0  681740634 2025-04-01 00:01:06 2025-04-16        0.0   

   interest_rate   loanStatus  totalDueAmount  
0            7.5  isCarryOver          6208.0  
1            7.5  isCarryOver             0.0  
2            7.5  isCarryOver             0.0  
Fetching time: 11.12 minutes


In [7]:
# Cell 6 - Clean + filter loans

# Columns with string dates
date_cols = ["loanDate", "dueDate"]
for col in date_cols:
    loans_df[col] = pd.to_datetime(loans_df[col], dayfirst=True, errors="coerce")

# Ensure new fields exist (fill with 0 if missing)
req_fields = ["dueAmount", "penaltyAmount"]
for col in req_fields:
    if col not in loans_df.columns:
        loans_df[col] = 0
    else:
        loans_df[col] = loans_df[col].fillna(0)

# Fill missing numeric values in the standard fields
num_cols = ["loanAmount",
        "amountPaid",
        "amountwithInterest",
        "interest_rate" ,
        "is_complete" , 
        "bonusAmount" ,
        "amountDisbursed", 
        "totalDueAmount"  ]
for col in num_cols:
    if col in loans_df.columns:
        loans_df[col] = loans_df[col].fillna(0)

# Optional: filter by dateTime window again (safety check)
loans_df = loans_df[
    (loans_df["dateTime"] >= date_start) &
    (loans_df["dateTime"] <= date_end)
]

# Sort like the boss does
loans_df = loans_df.sort_values("dateTime", ascending=True)

print("Cleaned loans in date window:", loans_df.shape)
print(loans_df.head(5))

Cleaned loans in date window: (170864, 14)
    loanDate  loanAmount  amountwithInterest  amountDisbursed  amountPaid  \
0 2025-04-01      1047.0              1126.0           1047.0         3.0   
1 2025-04-01      8000.0              8600.0           8000.0      9245.0   
2 2025-04-01     10000.0             10750.0          10000.0     11557.0   
3 2025-04-01      5000.0              5375.0           5000.0      5779.0   
4 2025-04-01      5000.0              5375.0           5000.0       181.0   

   bonusAmount   clientID            dateTime    dueDate  dueAmount  \
0       1126.0  677688025 2025-04-01 00:00:30 2025-04-15     1208.0   
1       8600.0  672635655 2025-04-01 00:00:56 2025-04-16        0.0   
2      10750.0  681740634 2025-04-01 00:01:06 2025-04-16        0.0   
3       5375.0  672667438 2025-04-01 00:01:10 2025-04-16        0.0   
4       5375.0  654218081 2025-04-01 00:01:24 2025-04-16     5598.0   

   interest_rate   loanStatus  totalDueAmount  penaltyAmount  
0   

In [8]:
# Cell 7 - Final cleaning + sample view for Loans

# Convert date fields properly
loans_df["loanDate"] = pd.to_datetime(loans_df["loanDate"], errors="coerce")
loans_df["dueDate"] = pd.to_datetime(loans_df["dueDate"], errors="coerce")

# Ensure numeric fields are clean
numeric_cols = ["loanAmount",
        "amountPaid",
        "amountwithInterest",
        "interest_rate" ,
        "bonusAmount" ,
        "amountDisbursed", 
        "totalDueAmount" ]
for col in numeric_cols:
    loans_df[col] = loans_df[col].fillna(0).apply(lambda x: max(x, 0))

# Clean text fields (IDs, status, refundType, etc.)
text_cols = [
        "loanStatus",
        "clientID"
    ]
for col in text_cols:
    if col in loans_df.columns:
        loans_df[col] = loans_df[col].astype(str).str.strip()

# Reset index for a neat table
loans_df.index = np.arange(loans_df.shape[0])

# Optional: sort by refund date (chronological order)
loans_df = loans_df.sort_values("loanDate", ascending=True)

# Show a random sample of fully cleaned refunds
print("Sample of fully cleaned refunds:")
display(loans_df.sample(10))


Sample of fully cleaned refunds:


,loanDate,loanAmount,amountwithInterest,amountDisbursed,amountPaid,bonusAmount,clientID,dateTime,dueDate,dueAmount,interest_rate,loanStatus,totalDueAmount,penaltyAmount
127312,2025-04-22,9000.0,9675.0,9000.0,9675.0,9675.0,653809773,2025-04-22 01:08:16,2025-05-07,726.0,7.5,isCarryOver,5726.0,0
77153,2025-04-13,1300.0,1398.0,1300.0,1398.0,1398.0,653126141,2025-04-13 01:09:59,2025-04-28,0.0,7.5,Normal,0.0,0
24754,2025-04-04,7000.0,7525.0,7000.0,7525.0,7525.0,682428584,2025-04-04 15:14:26,2025-04-19,0.0,7.5,Normal,0.0,0
42507,2025-04-07,3500.0,3763.0,3500.0,3763.0,3763.0,670307037,2025-04-07 08:12:42,2025-04-22,0.0,7.5,Normal,0.0,0
127857,2025-04-22,10000.0,10750.0,10000.0,10750.0,10750.0,680244196,2025-04-22 02:33:46,2025-05-07,0.0,7.5,Normal,0.0,0
66726,2025-04-11,1046.0,1125.0,1046.0,1125.0,1125.0,679236239,2025-04-11 07:50:25,2025-04-26,0.0,7.5,Normal,0.0,0
133951,2025-04-23,9500.0,10213.0,9500.0,102.0,10213.0,651826968,2025-04-23 04:34:06,2025-05-08,10877.0,7.5,isCarryOver,15877.0,0
147587,2025-04-25,10000.0,10750.0,10000.0,8.0,10750.0,673914865,2025-04-25 17:38:47,2025-05-10,11549.0,7.5,isCarryOver,16549.0,0
92325,2025-04-14,5340.0,5741.0,5340.0,6357.0,5741.0,673179467,2025-04-14 16:25:09,2025-04-29,0.0,7.5,isCarryOver,4815.0,0
109949,2025-04-18,3300.0,3548.0,3300.0,0.0,3548.0,670430981,2025-04-18 05:27:43,2025-05-03,3815.0,7.5,isCarryOver,8815.0,0


In [9]:
# Printing the payload csv
loans_df.to_csv("loans_april_2025.csv", index=False, sep=",")
print("csv export of loans_april_2025.csv complete")

csv export of loans_april_2025.csv complete
